In [1]:
import pandas as pd
import metadata_parser as mp
import requests
from bs4 import BeautifulSoup as bs
import re
import undetected_chromedriver as uc
import time

In [2]:
arxiv = pd.read_excel("./paper-links/PaperLinks_Arxiv.xlsx")

In [ ]:
# Get the paper metadata
list_year, list_title, list_conf_jour, list_authors, list_keywords, list_abstract = [], [], [], [], [], []
driver = uc.Chrome()
index = 1
for paper_url in arxiv["Paper Links"]:
    # Get Page
    driver.get(paper_url)
    time.sleep(5)
    
    # Get Year
    try:
        year_html = driver.find_element_by_xpath("//meta[@name='citation_date']")
        year = year_html.get_attribute("content").split("/")[0]
    except:
        year = "failed_to_scrap"
        
    # Get Title
    try:
        title_html = driver.find_element_by_xpath("//meta[@name='citation_title']")
        title = title_html.get_attribute("content")
    except:
        title = "failed_to_scrap"
        
    # Get Conf/Journal Name
    conf_jour_name = "arxiv_preprint"
    
    # Get Authors
    try:
        authors_html = driver.find_elements_by_xpath("//meta[@name='citation_author']")
        for i in range(0,len(authors_html)):
            authors_name = authors_html[i].get_attribute("content").split(", ")
            if i == 0:
                authors = authors_name[1]+" "+authors_name[0]
            else:
                authors = authors+", "+authors_name[1]+" "+authors_name[0]
    except:
        authors = "failed_to_scrap"
    
    # Get Keywords
    keywords = "no_keywords" # Arxiv does not provide keywords metadata like ACL
    
    # Get Abstract
    try:
        abstract_html = driver.find_element_by_xpath("//meta[@name='citation_abstract']")
        abstract = abstract_html.get_attribute("content")
        abstract = re.sub("\n","",abstract).strip()
    except:
        abstract = "failed_to_scrap"
    
    # Append to the list
    list_year.append(year)
    list_title.append(title)
    list_conf_jour.append(conf_jour_name)
    list_authors.append(authors)
    list_keywords.append(keywords)
    list_abstract.append(abstract)
    print(index)
    index += 1

driver.close()

In [4]:
# Concatenate list as new column
arxiv["Year"], arxiv["Title"], arxiv["Conf/Journal Name"] = list_year, list_title, list_conf_jour
arxiv["Authors"], arxiv["Keywords"], arxiv["Abstract"] = list_authors, list_keywords, list_abstract

In [6]:
arxiv.to_excel("./paper-metadata/PaperMetadata_Unclean_Arxiv.xlsx", index = False)